# 음악 플리에서 추천노래 갖고 오기

1. 설치

In [ ]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 4.8 MB/s eta 0:00:00


2. 토큰 발급

In [ ]:
from spotipy.oauth2 import SpotifyOAuth
from spotipy import Spotify

sp_oauth = SpotifyOAuth(
    client_id="본인 클라이언트ID 입력",
    client_secret="본인 클라이언트 비밀번호 입력",
    redirect_uri="리디렉션 URI",
    scope="playlist-modify-public playlist-modify-private user-read-currently-playing user-read-playback-state"
)

auth_url = sp_oauth.get_authorize_url()
print(f"링크 클릭 -> 리디렉션된 URL을 붙여 넣기(code:뒷부분 복붙):\n{auth_url}")

response_url = input("리디렉션된 전체 URL : ")
code = sp_oauth.parse_response_code(response_url)
token_info = sp_oauth.get_access_token(code)

sp = Spotify(auth=token_info['access_token'])


링크 클릭 -> 리디렉션된 URL을 붙여 넣기:
https://accounts.spotify.com/authorize?client_id=fd67c1837aba4051bef7fde0899e96cc&response_type=code&redirect_uri=http%3A%2F%2F127.0.0.1%3A8888%2Fcallback&scope=playlist-modify-public+playlist-modify-private+user-read-currently-playing+user-read-playback-state
리디렉션된 전체 URL : AQApFNRWI5aEsN_SN_3CXxMJnskF9yUqx3MTl95LuFr7vLBVWdw0_yNmy5dHtq8f_b792NnZR2RKhVnTxL9ll8QlUb2THTDg0PzCxVJ2wiZAOy-tPbMsrNCc8X5RM-e2zgVzCLCh-HsKFFDdawD4gjSWdlg8g14s3WjxEvlOwqzVDtG-RFlz4hxxWU5fStQDQ4a6Ljt0x1zrL62iX1cz0yKogPqfjvVe2cqQdOo1lEX4YB0D4NIDzzkzg41-Br0sVFmRWmiwJM2_S7n_PLFIStNKyNmzTVWw0Sg3avYqZrtk0h4dduA9iNKw


<ipython-input-2-7bfb1186908d>:16: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


3. main

In [5]:
import re

def is_korean(text):
    return bool(re.search('[\u3131-\u3163\uac00-\ud7a3]+', text))

def is_korean_track(track):
    return is_korean(track['name']) or is_korean(track['artists'][0]['name'])

INDIE_KEYWORDS = [
    # 🇰🇷 한국어 키워드
    '인디', '밴드', '자작곡', '싱어송라이터', '홍대', '언플러그드',
    '혼자', '어쿠스틱', '비주류', '마이너', '자유', '자유로운', '실험적',

    # 🇬🇧🇺🇸 영어 키워드
    'indie', 'band', 'diy', 'singer-songwriter', 'lofi', 'bedroom',
    'alternative', 'alt', 'underground', 'unplugged', 'acoustic',
    'experimental', 'non-mainstream', 'offbeat', 'authentic', 'raw'
]

def is_moody(track, playlists):
    name = track['name'].lower()
    artist = track['artists'][0]['name'].lower()
    combined_text = name + " " + artist
    for pl in playlists:
        combined_text += " " + pl.get('name', '').lower()
    return any(keyword in combined_text for keyword in INDIE_KEYWORDS)

def get_current_or_input_track(sp):
    current_track = sp.current_user_playing_track()
    if current_track is None or current_track['item'] is None:
        print("현재 재생 중인 곡이 없어요.")
        track_name = input("추천하고 싶은 곡 제목을 입력해주세요: ").strip()
        results = sp.search(q=track_name, type='track', limit=5)
        items = results['tracks']['items']

        if not items:
            print("해당 제목의 곡을 찾을 수 없어요.")
            exit()

        print("\n검색 결과:")
        for i, item in enumerate(items):
            print(f"{i+1}. {item['name']} - {item['artists'][0]['name']}")

        selected_index = input("\n👤 정확한 곡을 선택하려면 번호를 입력해주세요 (예: 1): ").strip()
        if not selected_index.isdigit() or not (1 <= int(selected_index) <= len(items)):
            print("잘못된 입력이에요. 숫자를 정확히 입력해 주세요.")
            exit()

        return items[int(selected_index) - 1]
    return current_track['item']

def search_playlists(sp, artist_name):
    playlist_results = sp.search(q=artist_name, type='playlist', limit=20)
    playlist_items = list(filter(None, playlist_results.get('playlists', {}).get('items', [])))

    def sort_priority(pl):
        title = pl.get('name', '').lower()
        return (0 if '인디' in title else 1, title)

    sorted_playlists = sorted(playlist_items, key=sort_priority)
    selected = sorted_playlists[:5]

    if selected:
        print("\n 선택된 플레이리스트:")
        for i, pl in enumerate(selected, 1):
            print(f"{i}. {pl['name']} (by {pl['owner']['display_name']})")
    else:
        print("관련 플레이리스트를 찾을 수 없어요. 인디 음악을 추천해드릴게요")

    return selected


def fetch_tracks_from_playlists(sp, playlists, exclude_id):
    all_tracks = []
    for pl in playlists:
        pid = pl['id']
        try:
            items = sp.playlist_tracks(pid)['items']
        except Exception as e:
            print(f" 플레이리스트 불러오기 실패: {e}")
            continue

        for item in items:
            track_info = item.get('track')
            if not track_info or track_info['id'] == exclude_id:
                continue
            all_tracks.append(track_info)

    unique_tracks = {t['id']: t for t in all_tracks}.values()
    filtered_tracks = [t for t in unique_tracks if t.get('popularity', 0) >= 3]
    return sorted(filtered_tracks, key=lambda x: x.get('popularity', 0))

def find_moody_korean_track(tracks, playlists):
    # 1순위: 인디 느낌 + 한국어
    for track in tracks:
        if is_korean_track(track) and is_moody(track, playlists):
            return track

    # 2순위: 인디 느낌 (한국어 아님)
    for track in tracks:
        if is_moody(track, playlists):
            return track

    # 3순위: 한국어 (인디 느낌 아님)
    for track in tracks:
        if is_korean_track(track):
            return track

     # 4순위: 아무거나 인기도 낮은 거라도
    if tracks:
        return tracks[0]  # popularity 기준으로 정렬되어 있으니까 첫 번째가 가장 인기 낮은 곡

    # 아무것도 없으면
    return None


def find_or_create_playlist(sp, user_id, name, description):
    playlists = sp.current_user_playlists()['items']
    for pl in playlists:
        if pl['name'] == name:
            return pl['id']
    new_pl = sp.user_playlist_create(user_id, name, False, description)
    return new_pl['id']

def save_to_playlist(sp, user_id, track):
    answer = input("\n 이 곡을 재생목록에 저장할까요? (Y/N): ").strip().lower()
    if answer == 'y':
        playlist_name = "고막친구🧸🎵"
        playlist_description = "고막친구가 추천한 인디 보석들 💎"
        playlist_id = find_or_create_playlist(sp, user_id, playlist_name, playlist_description)
        track_uri = f"spotify:track:{track['id']}"
        sp.playlist_add_items(playlist_id, [track_uri])
        print("추천곡이 재생목록에 저장됐어요!")
    else:
        print("저장하지 않았어요. 다음 추천곡도 기대해주세요!")

def main(sp):
    track = get_current_or_input_track(sp)
    track_name = track['name']
    artist_name = track['artists'][0]['name']
    track_id = track['id']

    print(f"\n 기준 트랙: {track_name} / 아티스트: {artist_name} / ID: {track_id}")

    selected_playlists = search_playlists(sp, artist_name)
    sorted_tracks = fetch_tracks_from_playlists(sp, selected_playlists, track_id)
    track_to_recommend = find_moody_korean_track(sorted_tracks, selected_playlists)

    print("\n 인디음악을 찾고 있어요 ..\n") # (인기도 낮은 한국어 곡을)

    if track_to_recommend:
        name = track_to_recommend['name']
        artist = track_to_recommend['artists'][0]['name']
        popularity = track_to_recommend.get('popularity', 'N/A')
        link = track_to_recommend['external_urls']['spotify']

        print(" 추천 곡 ")
        print(f"제목      : {name}")
        print(f"아티스트 : {artist}")
        print(f"인기도    : {popularity}")
        print(f"Spotify에서 듣기 -> {link} \n")

        user_id = sp.current_user()['id']
        save_to_playlist(sp, user_id, track_to_recommend)
    else:
        print("추천할 곡이 없어요. 다음엔 더 좋은 곡을 찾아볼게요!")


In [6]:
main(sp)

현재 재생 중인 곡이 없어요.
추천하고 싶은 곡 제목을 입력해주세요: 신호등

검색 결과:
1. Traffic light - Lee Mujin
2. Episode - Lee Mujin
3. Fry’s Dream - AKMU
4. MERRY-GO-ROUND (Feat. Zion.T, Wonstein) (Prod. Slom) - sokodomo
5. Traffic light - 박치용

👤 정확한 곡을 선택하려면 번호를 입력해주세요 (예: 1): 1

 기준 트랙: Traffic light / 아티스트: Lee Mujin / ID: 03qu1u4hDyepQQi2lNxCka

 선택된 플레이리스트:
1.  2024 Lee Mujin National Tour Concert <별책부록> Setlist (by Leemujin Global)
2. 2025 LEE MUJIN CONCERT [別冊附錄] in TAIPEI 250308 (by Y)
3. Lee Mujin Playlist (by tsumtsum)
4. LEE MUJIN PLAYLIST (by Shinggibanggi 💙)
5. 李茂珍 Lee Mujin  (by Ann)

 인디음악을 찾고 있어요 ..

 추천 곡 
제목      : 산책
아티스트 : Lee Mujin
인기도    : 28
Spotify에서 듣기 -> https://open.spotify.com/track/6NEGdk4Dp1Ic547RI80c6L 


 이 곡을 재생목록에 저장할까요? (Y/N): y
추천곡이 재생목록에 저장됐어요!
